# Анализ социально-экономических факторов на уровень счастья и удовлетворенности жизнью в странах Европы

## **1. Сбор данных**

Описание признаков: [ссылка](https://docs.google.com/document/d/16r0fiIdUSfEgPYzUKSXar4MYbYAaMIZl/edit)

Датасет в формате csv: [ссылка](https://drive.google.com/file/d/1ZFxt4HygAkduFb7HVVI3ZcIyf51oIXCA/view?usp=share_link)

In [18]:
import pandas as pd
import numpy as np

In [19]:
file = 'final_dataset.csv'
data = pd.read_csv(file)
data.head()

,Unnamed: 0,idno,agea,rlgdgr,polintr,sclmeet,happy,cntry,netustm,nwspol,...,actrolga,cptppola,trstprl,trstlgl,trstplc,trstplt,vote,stfeco,stfdem,stfedu
0,0,50014,65,5,1,4,8,AT,180,90,...,5,5,6,9,10,5,1,6,6,8
1,1,50030,21,0,2,7,9,AT,570,90,...,2,2,6,6,4,1,1,2,7,10
2,2,50057,53,8,2,4,9,AT,30,30,...,4,3,7,5,8,4,1,6,6,5
3,3,50106,78,6,3,6,7,AT,6666,15,...,2,3,5,6,9,3,2,4,6,5
4,4,50145,64,1,2,5,9,AT,120,60,...,1,3,6,8,8,5,1,6,8,9


In [35]:
data.shape

(40156, 53)

In [21]:
print("Количество пропущенных значений в 'health':", data['health'].isnull().sum())
print("Количество пропущенных значений в 'sclmeet':", data['sclmeet'].isnull().sum())
print("Количество пропущенных значений в 'atchctr':", data['health'].isnull().sum())
print("Количество пропущенных значений в 'aesfdrk':", data['health'].isnull().sum())
print("Количество пропущенных значений в 'dscrgrp':", data['health'].isnull().sum())
print("Количество пропущенных значений в 'pray':", data['health'].isnull().sum())
print("Количество пропущенных значений в 'hhmmb':", data['health'].isnull().sum())
print("Количество пропущенных значений в 'gndr':", data['health'].isnull().sum())
print("Количество пропущенных значений в 'yrbrn':", data['health'].isnull().sum())
print("Количество пропущенных значений в 'impdiffa':", data['health'].isnull().sum())
print("Количество пропущенных значений в 'impfuna':", data['health'].isnull().sum())

Количество пропущенных значений в 'health': 0
Количество пропущенных значений в 'sclmeet': 0
Количество пропущенных значений в 'atchctr': 0
Количество пропущенных значений в 'aesfdrk': 0
Количество пропущенных значений в 'dscrgrp': 0
Количество пропущенных значений в 'pray': 0
Количество пропущенных значений в 'hhmmb': 0
Количество пропущенных значений в 'gndr': 0
Количество пропущенных значений в 'yrbrn': 0
Количество пропущенных значений в 'impdiffa': 0
Количество пропущенных значений в 'impfuna': 0


In [22]:
# Subjective general health
bad_answers = data['health'].isin([7, 8, 9])
data.loc[bad_answers, 'health'] = np.nan
print("[health]Количество null'ов:", data['health'].isnull().sum())

[health]Количество null'ов: 53


In [24]:
# How often socially meet with friends, relatives or colleagues
bad_answers = data['sclmeet'].isin([77, 88, 99])
data.loc[bad_answers, 'sclmeet'] = np.nan
print("[sclmeet]Количество null'ов:", data['sclmeet'].isnull().sum())

[sclmeet]Количество null'ов: 75


In [25]:
# How emotionally attached to [country]
bad_answers = data['atchctr'].isin([77, 88, 99])
data.loc[bad_answers, 'atchctr'] = np.nan
print("[atchctr]Количество null'ов:", data['atchctr'].isnull().sum())

[atchctr]Количество null'ов: 165


In [26]:
# Feeling of safety of walking alone in local area after dark
bad_answers = data['aesfdrk'].isin([7, 8, 9])
data.loc[bad_answers, 'aesfdrk'] = np.nan
print("[aesfdrk]Количество null'ов:", data['aesfdrk'].isnull().sum())

[aesfdrk]Количество null'ов: 295


In [27]:
# Member of a group discriminated against in this country
data['dscrgrp'] = data['dscrgrp'].replace({2: 0, 7: np.nan, 8: np.nan, 9: np.nan})
print("[dscrgrp]Количество null'ов:", data['dscrgrp'].isnull().sum())

[dscrgrp]Количество null'ов: 270


In [28]:
# How often pray apart from at religious services
bad_answers = data['pray'].isin([77, 88, 99])
data.loc[bad_answers, 'pray'] = np.nan
print("[pray]Количество null'ов:", data['pray'].isnull().sum())

[pray]Количество null'ов: 748


In [29]:
# Number of people living regularly as member of household
bad_answers = data['hhmmb'].isin([77, 88, 99])
data.loc[bad_answers, 'hhmmb'] = np.nan
print("[hhmmb]Количество null'ов:", data['hhmmb'].isnull().sum())

[hhmmb]Количество null'ов: 219


In [30]:
# Gender
data['gndr'] = data['gndr'].replace({2: 0, 9: np.nan})
print("[gndr]Количество null'ов:", data['gndr'].isnull().sum())

[gndr]Количество null'ов: 0


In [31]:
# And in what year were you born?
bad_answers = data['yrbrn'].isin([7777, 8888, 9999])
data.loc[bad_answers, 'yrbrn'] = np.nan
print("[yrbrn]Количество null'ов:", data['yrbrn'].isnull().sum())

[yrbrn]Количество null'ов: 270


In [33]:
# Important to try new and different things in life
bad_answers = data['impdiffa'].isin([66, 77, 88, 99])
data.loc[bad_answers, 'impdiffa'] = np.nan
print("[impdiffa]Количество null'ов:", data['impdiffa'].isnull().sum())

[impdiffa]Количество null'ов: 702


In [34]:
# Important to seek fun and things that give pleasure
bad_answers = data['impfuna'].isin([66, 77, 88, 99])
data.loc[bad_answers, 'impfuna'] = np.nan
print("[impfuna]Количество null'ов:", data['impfuna'].isnull().sum())

[impfuna]Количество null'ов: 707
